In [1]:
import manga109api
import random

In [4]:
manga109_root_dir = "D:\RESEARCH PROJECT\Data\Manga109"
p = manga109api.Parser(root_dir=manga109_root_dir)

In [61]:
print(p.books)

['ARMS', 'AisazuNihaIrarenai', 'AkkeraKanjinchou', 'Akuhamu', 'AosugiruHaru', 'AppareKappore', 'Arisa', 'BEMADER_P', 'BakuretsuKungFuGirl', 'Belmondo', 'BokuHaSitatakaKun', 'BurariTessenTorimonocho', 'ByebyeC-BOY', 'Count3DeKimeteAgeru', 'DollGun', 'Donburakokko', 'DualJustice', 'EienNoWith', 'EvaLady', 'EverydayOsakanaChan', 'GOOD_KISS_Ver2', 'GakuenNoise', 'GarakutayaManta', 'GinNoChimera', 'Hamlet', 'HanzaiKousyouninMinegishiEitarou', 'HaruichibanNoFukukoro', 'HarukaRefrain', 'HealingPlanet', 'HeiseiJimen', 'HighschoolKimengumi_vol01', 'HighschoolKimengumi_vol20', 'HinagikuKenzan', 'HisokaReturns', 'JangiriPonpon', 'JijiBabaFight', 'Joouari', 'Jyovolley', 'KarappoHighschool', 'KimiHaBokuNoTaiyouDa', 'KoukouNoHitotachi', 'KuroidoGanka', 'KyokugenCyclone', 'LancelotFullThrottle', 'LoveHina_vol01', 'LoveHina_vol14', 'MAD_STONE', 'MadouTaiga', 'MagicStarGakuin', 'MagicianLoad', 'MariaSamaNihaNaisyo', 'MayaNoAkaiKutsu', 'MemorySeijin', 'MeteoSanStrikeDesu', 'MiraiSan', 'MisutenaideDaisy'

In [62]:
# Code to get all faces from manga and save them into the correct foler.

import manga109api
from PIL import Image, ImageDraw
#! ["body", "face", "frame", "text"] 
def draw_rectangle(img, x0, y0, x1, y1, annotation_type):
    if annotation_type in ["face"]:
    # assert annotation_type in ["body", "face", "frame", "text"]
        color = {"body": "#258039", "face": "#f5be41",
             "frame": "#31a9b8", "text": "#cf3721"}[annotation_type]
        draw = ImageDraw.Draw(img)
        draw.rectangle([x0, y0, x1, y1], outline=color, width=4)

def get_faces(book_title):
    manga109_root_dir = "D:\RESEARCH PROJECT\Data\Manga109"
    j = 0

    p = manga109api.Parser(root_dir=manga109_root_dir)
    annotation = p.get_annotation(book=book_title)

    for i in range(0, len(annotation['page'])):

        img = Image.open(p.img_path(book=book_title, index=i))

        for annotation_type in ["face"]:
            rois = annotation["page"][i][annotation_type]
            for roi in rois:
                j += 1
                # draw_rectangle(img, roi["@xmin"], roi["@ymin"], roi["@xmax"], roi["@ymax"], annotation_type)
                if not os.path.exists(f"../../data/manga109_faces/{book_title}"):
                    os.makedirs(f"../../data/manga109_faces/{book_title}")


                img2 = img.crop((roi["@xmin"], roi["@ymin"], roi["@xmax"], roi["@ymax"]))
                img2.save(f"../../data/manga109_faces/{book_title}/{j}.jpg")

    # img.save("out.jpg")

In [63]:
for book in p.books:
    get_faces(book)

In [65]:
# Code to get random images from a manga page as training data, no faces.

import manga109api
from PIL import Image, ImageDraw
def get_random_frames(book_title):
    manga109_root_dir = "D:\RESEARCH PROJECT\Data\Manga109"
    j = 0

    p = manga109api.Parser(root_dir=manga109_root_dir)
    annotation = p.get_annotation(book=book_title)

    for i in range(0, len(annotation['page'])):

        img = Image.open(p.img_path(book=book_title, index=i))
        
        x_min = []
        y_min = []
        for annotation_type in ["text"]:
            rois = annotation["page"][i][annotation_type]
            for roi in rois:
                # texts_list.append({'xmin': roi["@xmin"], 'ymin': roi["@ymin"], 'xmax': roi["@xmax"], 'ymax': roi["@ymax"]})
                x_min.append(roi["@xmin"])
                y_min.append(roi["@ymin"])
        # print(texts_list)

        for annotation_type in ["face"]:
            rois = annotation["page"][i][annotation_type]
            for roi in rois:
                j += 1
                # draw_rectangle(img, roi["@xmin"], roi["@ymin"], roi["@xmax"], roi["@ymax"], annotation_type)
                if not os.path.exists(f"../../data/manga109_random_frames/{book_title}"):
                    os.makedirs(f"../../data/manga109_random_frames/{book_title}")


                xmin = random.randint(1, img.size[0] - 110)
                
                ymin = random.randint(1, img.size[1] - 110)
                
                takeClosest = lambda num,collection:min(collection,key=lambda x:abs(x-num), default=0)

                to_break = 0
                can_save = True 

                while abs(xmin - int(takeClosest(xmin, x_min))) < 110 or abs(ymin - int(takeClosest(ymin, y_min))) < 110:
                    xmin = random.randint(1, img.size[0] - 110)
                    ymin = random.randint(1, img.size[1] - 110)
                    to_break += 1
                    if to_break == 5:
                        can_save = False
                        break
                
                   

                xmax = xmin + 100
                ymax = ymin + 100
                #print(img.size)
                #print(xmin, ymin, xmax, ymax)
                if can_save == True:
                    img2 = img.crop((xmin, ymin, xmax, ymax))
                    img2.save(f"../../data/manga109_random_frames/{book_title}/{j}.jpg")

    # img.save("out.jpg")

In [66]:
for book in p.books:
    get_random_frames(book)

In [67]:
# Code to get the speech bubbles from my dataset.

import manga109api
from PIL import Image, ImageDraw
#! ["body", "face", "frame", "text"] 
def get_texts(book_title):
    manga109_root_dir = "D:\RESEARCH PROJECT\Data\Manga109"
    j = 0

    p = manga109api.Parser(root_dir=manga109_root_dir)
    annotation = p.get_annotation(book=book_title)

    for i in range(0, len(annotation['page'])):

        img = Image.open(p.img_path(book=book_title, index=i))
        
        for annotation_type in ["text"]:
            rois = annotation["page"][i][annotation_type]
            for roi in rois:
                j += 1
                # draw_rectangle(img, roi["@xmin"], roi["@ymin"], roi["@xmax"], roi["@ymax"], annotation_type)
                if not os.path.exists(f"../../data/manga109_text/{book_title}"):
                    os.makedirs(f"../../data/manga109_text/{book_title}")

                img2 = img.crop((roi["@xmin"], roi["@ymin"], roi["@xmax"], roi["@ymax"]))
                img2.save(f"../../data/manga109_text/{book_title}/{j}.jpg")

In [68]:
for book in p.books:
    get_texts(book)

SystemError: tile cannot extend outside image

In [7]:
# This is some code to create the YOLO training data that I will need for my YOLO model.

import manga109api
from PIL import Image, ImageDraw

def draw_rectangle(img, x0, y0, x1, y1, annotation_type):
    if annotation_type in ["face"]:
    # assert annotation_type in ["body", "face", "frame", "text"]
        color = {"body": "#258039", "face": "#f5be41",
             "frame": "#31a9b8", "text": "#cf3721"}[annotation_type]
        draw = ImageDraw.Draw(img)
        draw.rectangle([x0, y0, x1, y1], outline=color, width=4)

def get_faces_for_yolo():
    manga109_root_dir = "D:\RESEARCH PROJECT\Data\Manga109"
    p = manga109api.Parser(root_dir=manga109_root_dir)
    j = 0
    for book_title in p.books:
        annotation = p.get_annotation(book=book_title)

        for i in range(0, len(annotation['page'])):
            
            img = Image.open(p.img_path(book=book_title, index=i))

            for annotation_type in ["face"]:
                j += 1
                rois = annotation["page"][i][annotation_type]
                f = open(f"../../data/manga109_yolo_faces/{j}.txt","w+")  
                for roi in rois:
                    
                    # draw_rectangle(img, roi["@xmin"], roi["@ymin"], roi["@xmax"], roi["@ymax"], annotation_type)

                    # w = ((roi["@xmax"] - roi["@xmin"]) / 2)
                    # h = ((roi["@ymax"] - roi["@ymin"]) / 2)
                    w = (roi["@xmax"] - roi["@xmin"])
                    h = (roi["@ymax"] - roi["@ymin"])
                    x = roi['@xmin'] + ((roi["@xmax"] - roi["@xmin"]) / 2)
                    y = roi["@ymin"] + ((roi["@ymax"] - roi["@ymin"]) / 2)

                    # print(img.size)

                    x = round(x / img.size[0], 6)
                    y = round(y / img.size[1], 6)
                    w = round(w / img.size[0], 6)
                    h = round(h / img.size[1], 6)

                    # print(x, y, w, h)
                

                    # print(roi["@xmin"], roi["@ymin"], roi["@xmax"], roi["@ymax"])
                    #img2 = img.crop((roi["@xmin"], roi["@ymin"], roi["@xmax"], roi["@ymax"]))
                    f.write(f"1 {x} {y} {w} {h}\n") 
                f.close()  
                img.save(f"../../data/manga109_yolo_faces/{j}.jpg")


In [8]:
get_faces_for_yolo()